# Air pump failure analysis

Following statements are deduced from given task `README.md`.

### Important observations
- sharp start and end of pressure (normally stable)
- slow start, end or pressure drops could mean pump failure
- typical air pump failure due to pressure drop in the first half of the cycle
- pressure is saved as a time series (ordered, same intervals)

### Data structure 
- MachineId, MeasurementId are Id's of machine and cycle
- Pressure (kPa)
- PumpFailed, SlowStart, SlowEnd (bool features)

### Task
- develop predictive model from `PumpFailed` and measure performance
- explain model

# Basic observations

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk

INDEXES = ['MachineId', 'MeasurementId']
TARGET = ['PumpFailed']

RANDOM_STATE = 42

In [ ]:
def print_unique_values(df: pd.DataFrame):
    """ Print unique values for each feature in given dataframe """
    print('Unique values:')
    for feature in df.columns:
        print(f"{feature:15} : {len(df[feature].unique())}")

## Labels dataframe

In [ ]:
df_labels = pd.read_csv('data/labels.csv')

display(df_labels.head())
display(df_labels.info())
display(df_labels.isna().sum())

# nan values in rows with measurementid = -1
# I could rename machineid with new numeric counter or just use category type
# I could drop rows with NaN values due to quite large amount of data 
#   - with more time I could try to replace it with most frequent values, median from distribution estimate

In [ ]:
print_unique_values(df_labels)

print()
print('Biggest MeasurementId: ', df_labels['MeasurementId'].max())

# 556 uniquq machines and 8836 unique measurement cycles
# bool features has nan, false, true -> drop
# biggest measurementid is 8834 which should fit into int16 type for memory saving

In [ ]:

# drop NaN and change types
df_labels = df_labels.dropna().astype({
    'MachineId' : 'category',
    'MeasurementId' : 'int16',
    'PumpFailed' : 'bool',
    'SlowStart' : 'bool',
    'SlowEnd' : 'bool'
})

display(df_labels.info())

# check target value counts
print(df_labels['PumpFailed'].value_counts())

# data with target value is inbalanced and binary, so I could just use prediction from Bernoulli distribution to create baseline model
# inbalanced dataset could create biased model for the PumpFailed = False, so to solve this I can:
#   - choose appropriate metric for validation (accuracy wont do in this case)
#   - create completely new dataset using bootstrapping (harder to compute, but could be implemented if more time)
#   - drop rows with PumpFailed = False to balance row counts (not optimal due to considerable dataset size loss)

In [ ]:
# pump bool feature combinations (all: 8)

df_labels[['PumpFailed', 'SlowStart', 'SlowEnd']].drop_duplicates()

# data does not contain any pumps with states:
# (true, true, true) -> pump failed with both slow start and end
# (true, true, false) -> pump failed only with slow start

# later I can plot pressure in time for each of these combinations

## Data dataframe

In [ ]:
df_data = pd.read_parquet('data/data.parquet', engine='fastparquet')

In [ ]:
display(df_data.head())
display(df_data.info())
display(df_data.isna().sum())

In [ ]:
print_unique_values(df_data)

print()
print('Largest pressure (kPa): ', df_data['Pressure'].max())

In [ ]:
df_data = df_data.dropna().astype({
    'MachineId' : 'category',
    'MeasurementId' : 'int16',
    'Pressure' : 'float32',
})

display(df_data.dtypes)

In [ ]:
# merge datasets using MachineId and MeasurementId
df_merged = pd.merge(df_data, df_labels, on=INDEXES)
df_merged.head()

## Merged dataframe pivoting

The dataframe is currently in merged state with ordered ressure measurements.
What I'll try to do now, is to pivot that dataframe, so that measured pressure values are shown ordered by time, but each column will represent unique measurement.

With that format I could manipulate easily with pressure measurements as time series.

In [ ]:
# select both IDs and pressure features and set "categorical" dataframe index (set_index does not change order of rows)
df_measurements = df_merged.loc[:, ['MachineId', 'MeasurementId', 'Pressure']].set_index(INDEXES)
df_measurements.head()

In [ ]:
# create new column time where value is incremented by one when pressure belongs to same indexes
# example: MachineId = 0_0_0 and MeasurementId = 0 will have increasing time series until it finds new Machine or Measurement ID 
df_measurements['Time'] = df_measurements.groupby(level=INDEXES, observed=False).cumcount()
df_measurements.head()

In [ ]:
# now it is just pivoting the table and replacing all shorter measurement sequences NaN values with zero 
df_measurements = df_measurements.pivot_table(index=INDEXES, columns='Time', values='Pressure', aggfunc='first', fill_value=0)
display(df_measurements.head())

assert df_measurements.shape[0] == df_labels.shape[0] # row count sanity check

In [ ]:
# now I'll just merge it with dataframe containing bool features and target value
df_all = pd.merge(df_labels.set_index(INDEXES), df_measurements, left_index=True, right_index=True).reset_index()

# Exploratory data analysis

In [ ]:
# test correlation between target and binary features
df_labels[['PumpFailed', 'SlowStart', 'SlowEnd']].corr().round(2)

# from correlation table I can't say that data are correlated or not
# note: correlation is not the best test for binary features and there should be also tested if data are from norm. distribution,
# but it could still be good indicator for doing more testing

In [ ]:
# beacause of only 6 bool feature combinations, I will plot example presure in time data for each of them 

# plot config
xlim, ylim = (50, 350), (-0.1, 1.7)
fig, axes = plt.subplots(3, 2, figsize = (14, 6))
fig.tight_layout()

# get possible combinations
combinations = df_all[['PumpFailed', 'SlowStart', 'SlowEnd']].drop_duplicates()

# loop for each bool feature combination
for i, [_ , [fail, start, end]] in enumerate(combinations.iterrows()):
    
    # get first pump matching feature combination
    first_pump_match = df_all[(df_all['PumpFailed'] == fail) & (df_all['SlowStart'] == start) & (df_all['SlowEnd'] == end)].iloc[0]
    
    # get machine and measurement id
    machine, measurement = first_pump_match['MachineId'], first_pump_match['MeasurementId']

    # get pressure series for given machine and cycle
    series_pressure = df_all[(df_all['MachineId'] == machine) & (df_all['MeasurementId'] == measurement)].iloc[0, 5:].reset_index(drop=True)

    # plot pressure in time for each combination
    ax = axes.ravel()[i]
    ax.plot(series_pressure, color='r' if fail else 'b')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_title(f"{'SlowStart' if start else ''} {'SlowEnd' if end else ''}")

plt.show()

# understanding data based on all possible boolean combinations

# predictions based on data observation:
# pump might fail due to fast drop at the beginning
# slow pressure decrease at the end could also mean pump failure
# low pressure could not necessarly mean pump failure (could be more about the pressure change speed) 

# Feature engineering

In [ ]:
# due to the data being time series, it would be probably better to create features such as:
# how long was pump running for
# pressure mean, variance, max
# pressure changes in time like:
#   - largest drop, largest spike
#   - time series decomposition (trend, seasonality, residual) more about actuasl pressure prediction

# then it could help to standardize, normalize data before doing new feature extraction

df_pressure = df_all.iloc[:, 5:]

In [ ]:
# as I said, I want to create features as largest drop, spike, etc. so it would be best to strip actual measured sequence
# I'll create function which will return starting and ending index based on where pressure is not zero
# then I will drop all unnecessary columns based on the smallest starting index and largest ending index
# this will produce same measured windows for each measured cycle but without too many zeroes

ZERO_SEQ = (-1, -1)

def get_nonzero_seq(df: pd.DataFrame):
    """ Get nonzero sequence (start, end) indexes from each row of given dataframe | (-1, -1) for full empty sequences """
    seq = []

    for _, row in df.iterrows():
        indexes = row[row > 0].index

        if len(indexes):
            seq.append((indexes[0], indexes[-1]))
        else:
            seq.append((-1, -1))

    return seq

seq = get_nonzero_seq(df_pressure)
filter_seq = list(filter(lambda x: x != ZERO_SEQ, seq))

seq_start, seq_end = min([x[0] for x in filter_seq]), max([x[1] for x in filter_seq])

print(f"Earliest pump activity (time): {seq_start}")
print(f"Latest pump activity (time):   {seq_end}")

# largest sequence start on 34 and ends on 1065 (not necessary the same measurement cycle)
# I'll add like 10 time ticks as a reserve to have at least some zero pressure data in the largest measurement cycles

In [ ]:
# drop unnecessary zeroes and reindex columns from zero
reserve = 10

df_pressure = df_pressure.iloc[:, seq_start - reserve:seq_end + reserve]
df_pressure.columns = range(len(df_pressure.columns))
df_pressure.shape

In [ ]:
# now I am finally going to create some new features from cycle measurements

cycle_length = pd.Series([0 if idx == ZERO_SEQ else idx[1] - idx[0] for idx in seq])
mean = df_pressure.mean(axis=1)
var = df_pressure.var(axis=1)
max_pressure = df_pressure.max(axis=1)

print(f"Min/Max measurement cycle length: {cycle_length.min()}, {cycle_length.max()}")
print(f"Measurement cycle mean, var, std: {cycle_length.mean():.3f}, {cycle_length.var():.3f}, {cycle_length.std():3f}")

# those are the simple features
# min measurement cycle length is 0 which is not optimal, but with given time I'll skip it for now
# also mean of cycle length is ~205.6, so I could create new features based on longer cycle or to drop outliers based

In [ ]:
# now for the pressure change features
# I could try to create largest pressure drop or spike in raw pressure value or in precentage
# both should be ok, but in this case I'll use percentage
# one thing which could be problematic is to take this change from the whole sequence, because of the inf, or 100% drop values
# because of that I will create more slices using the reserve previously used for letting start and end zeroes  


# now I'll create new features dataframe and then I'll concat it with the original needed features
new_features = pd.DataFrame({
    'Length' : cycle_length,
    'Mean' : mean,
    'Var' : var,
    'MaxPressure': max_pressure,
})

new_features.head()

In [ ]:
df_all_new_features = pd.concat((df_all[['PumpFailed', 'SlowStart', 'SlowEnd']], new_features), axis=1)
# df_all_new_features = df_all_new_features.dropna()

In [ ]:
df_all_new_features

# Model construction

## Base data

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.pipeline import Pipeline

In [ ]:
# used model config
models = {
    'LogisticRegression' : {
        'model' : sk.linear_model.LogisticRegression(random_state=RANDOM_STATE, max_iter=500),
        'params' : {
            'model__C' : np.linspace(1e-3, 1e3, 20)
        }
    },
    'RandomForestClassifier' : {
        'model' : sk.ensemble.RandomForestClassifier(),
        'params' : {
            'model__n_estimators' : [50, 100, 200],
        }
    },
}

In [ ]:
X, y = df_labels.drop(TARGET + INDEXES, axis=1), np.ravel(df_labels.loc[:, TARGET])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

print('Train:', X_train.shape)
print('Test:', X_test.shape)

In [ ]:
best_models = {}

# run gridsearchCV to tune hyperparams
for model_name, conf in models.items():
    model, params = conf['model'], conf['params']

    pipe = Pipeline([('model', model)])

    grid_search = sk.model_selection.GridSearchCV(pipe, params, scoring=make_scorer(f1_score, greater_is_better=True), cv=5)
    grid_search.fit(X_train, y_train)   

    best_models[model_name] = {
        'model': grid_search.best_estimator_, 
        'params': grid_search.best_params_, 
        'score' : grid_search.best_score_
    }

In [ ]:
# get best type of model after hyperparam tuning
model = max([(conf['score'], conf['model']) for conf in best_models.values()], key=lambda x: x[0])[1]
# model.fit(X_train, y_train)

display(model)

print(f"f1 score (train):  {f1_score(y_train, model.predict(X_train)):.3f}")
print(f"f1 score (test):   {f1_score(y_test, model.predict(X_test)):.3f}")

print()

print(f"acc score (train): {accuracy_score(y_train, model.predict(X_train)):.3f}")
print(f"acc score (test):  {accuracy_score(y_test, model.predict(X_test)):.3f}")

## New features

In [ ]:
X, y = df_all_new_features.drop(TARGET, axis=1), np.ravel(df_all_new_features.loc[:, TARGET])
X_train, X_test, y_train, y_test = tt_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

print('Train:', X_train.shape)
print('Test:', X_test.shape)

In [ ]:
ct = sk.compose.ColumnTransformer([
    ('OneHot', sk.preprocessing.OneHotEncoder(), ['SlowStart', 'SlowEnd']),
    ('MinMax', sk.preprocessing.MinMaxScaler(), ['MaxPressure']),
    ('Scaler', sk.preprocessing.StandardScaler(), ['Length', 'MaxPressure'])
], remainder='passthrough')

In [ ]:
best_models = {}

# run gridsearchCV to tune hyperparams
for model_name, conf in models.items():
    model, params = conf['model'], conf['params']

    pipe = Pipeline([
        ('prep', ct),
        ('model', model)
    ])

    grid_search = sk.model_selection.GridSearchCV(pipe, params, scoring=make_scorer(f1_score, greater_is_better=True), cv=5, verbose=True)
    grid_search.fit(X_train, y_train)   

    best_models[model_name] = {
        'model': grid_search.best_estimator_, 
        'params': grid_search.best_params_, 
        'score' : grid_search.best_score_
    }

In [ ]:
model = max([(conf['score'], conf['model']) for conf in best_models.values()], key=lambda x: x[0])[1]

model.fit(X_train, y_train)

display(model)

print(f"f1 score (train):  {f1_score(y_train, model.predict(X_train)):.3f}")
print(f"f1 score (test):   {f1_score(y_test, model.predict(X_test)):.3f}")

print()

print(f"acc score (train): {accuracy_score(y_train, model.predict(X_train)):.3f}")
print(f"acc score (test):  {accuracy_score(y_test, model.predict(X_test)):.3f}")

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, model.predict(X_test), labels=model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['OK', 'Failure'])

disp.plot()
plt.title('Pump status confusion matrix')
plt.show()